In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
import os

import datetime

import pandas as pd

from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

In [ ]:
service = ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.maximize_window()

# เก็บข้อมูลว่าแต่ละจังหวัดมี no. อะไร ตามที่ตั้ง no. ให้ใน file data prep และเก็บพิกัดของเว็บ Ventusky ที่จะ render map บริเวณจังหวัดนั้น ๆ

In [ ]:
provinceDict = {"yala":[74,"6.506;101.341;9"],"satun":[71,"6.680;100.403;9"], "songkhla":[72,"7.125;100.658;9"],"trang":[63,"7.493;99.926;9"], \
"nakhon-si-thammarat":[65-1,"8.227;99.789;9"],"wichit":[70-1,"8.038;98.722;9"],"surat-thani":[74-1,"9.115;99.007;9"],"ranong":[71-1,"9.863;98.867;9"], \
"kanchanaburi":[57-1,"13.923;100.349;9"] , "samut-sakhon":[44-1,"13.931;100.267;9"], "bangkok":[76-1,"13.931;100.267;9"] , "samut-prakan":[42-1,"13.931;100.267;9"] , \
"mueang-nonthaburi":[35-1,"13.931;100.267;9"] , \
"pathum-thani":[36-1,"13.931;100.267;9"] , "phra-nakhon-si-ayutthaya" : [37-1,"13.931;100.267;9"]  , "saraburi":[48-1,"14.475;100.437;9"] , \
"lop-buri":[41-1,"14.475;100.437;9"] , "sing-buri":[45-1,"14.475;100.437;9"] , "prachin-buri":[54-1,"14.475;100.437;9"] , "nakhon-nayok":[32-1,"14.475;100.437;9"] , \
"chiang-rai":[1-1, "19.22;99.99;8"], "mae-hong-son":[6-1, "19.02;97.87;8"], "phayao":[4-1, "19.10;100.27;8"], "nan":[3-1, "18.79;100.80;8"], \
"chiang-mai":[2-1, "18.93;99.06;8"], "lamphun":[8-1, "18.50;99.07;8"], "lampang":[7-1, "18.35;99.42;8"], "phrae":[5-1, "18.17;100.26;8"], "uttaradit":[9-1, "17.55;100.10;8"], \
"sukhothai":[46-1, "16.96;99.97;8"], "tak":[58-1, "16.83;99.18;8"], "phitsanulok":[39-1, "16.83;100.20;8"], "kamphaeng-phet":[30-1, "16.35;99.30;8"], "phichit":[38-1, "16.37;100.45;8"], \
"phetchabun":[40-1, "16.50;101.39;8"], "nakhon-sawan":[34-1, "15.73;100.26;8"], "uthai-thani":[50-1, "15.35;100.12;8"], "nong-khai":[25-1, "17.91;102.82;8"], \
"loei":[21-1, "17.47;101.90;8"], "udon-thani":[27-1, "17.50;102.79;8"], "nakhon-phanom":[13-1, "17.41;104.73;8"], \
"sakon-nakhon":[22-1, "17.17;104.19;8"], "khon-kaen":[11-1, "16.34;102.69;8"], "kalasin":[10-1, "16.50;103.73;8"], \
"roi-et":[20-1, "16.10;103.69;8"],  "chaiyaphum":[12-1, "15.87;102.05;8"], \
"ubon-ratchathani":[28-1, "14.98;105.17;8"], "buriram":[16-1, "14.84;102.88;8"], \
"nakhon-ratchasima":[14-1,"14.85;102.04;8"], "chon-buri":[77-1, "13.25;101.13;8"], "rayong":[55-1, "12.67;101.21;8"], "trat":[52, "12.25;102.55;8"]}

In [ ]:
print(len(provinceDict))

# Function scrape ข้อมูลชองทุกจังหวัดใน 1 วันที่ระบุ

In [ ]:
def findStuff(date,type):
    firstTime=True
    scrapedValue = []
    for key,value in provinceDict.items():
        url = f'https://www.ventusky.com/?p={value[1]}&l={type}&t={date}/0600'
        driver.get(url=url) 
        #wait for web to load
        if(firstTime):
            time.sleep(2)
            firstTime=False
        try:
            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, f"a[href*={key}]")))
            print ("Page is ready!")
        except TimeoutException:
            print ("Loading took too much time!")
        ozoneValue = driver.find_elements_by_css_selector(f'a[href={key}]')[0]
        driver.execute_script("arguments[0].click();", ozoneValue)
        try:
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, f'a[class="qo am zh"][href="{key}"]')))
        except:
            print("not found href 1")
            
        time.sleep(1)
        ozoneValue = driver.find_elements_by_css_selector(f'a[href={key}]')
        realOzoneValue=ozoneValue[-1]
        for e in ozoneValue:
            testValue = e.get_attribute("data-value")
            if(testValue!=None):
                realOzoneValue=e
                break
            else:
                print(testValue)
        scrapedValue.append([value[0],realOzoneValue.get_attribute('data-value').split(" ")[0]])
        print(realOzoneValue.get_attribute('data-value').split(" ")[0])
    return scrapedValue
    

# Function return วันถัดไปที่จะทำการ scrape

In [ ]:
yearList = ["2020","2021","2022"]
monthList = ["01","02","03","04","05","06","07","08","09","10","11","12"]
dayList = ["01","08","15","22"]
def nextDay(date):
    year=date[0:4]
    month=date[4:6]
    day=date[6:8]
    if day =="22":
        day=dayList[0]
        if month != "12":
            month=monthList[monthList.index(month)+1]
        else:
            month="01"
            year=yearList[yearList.index(year)+1]
    else:
        day=dayList[dayList.index(day)+1]
    return year+month+day

# Scrape ข้อมูล CO

In [ ]:
df_struct = {
    "date":[],
    "provinceNo":[],
    "co":[]
}

dfCO = pd.DataFrame(df_struct)
coResult = {}

In [ ]:
initialDate = "20200722"

date = initialDate

while date <= "20220122":
    scrapedValue2 = findStuff(date,"co")
    print(scrapedValue2)
    coResult[date] = scrapedValue2

    df = pd.DataFrame(
    data=scrapedValue2,
    columns=['provinceNo','co']
    )
    df['date'] = date
    first_column = df.pop('date')
    df.insert(0, 'date', first_column)
    dfCO = pd.concat([dfCO,df])

    date = nextDay(date)

# Export ข้อมูล CO ที่ scrape

In [ ]:
dfCO.to_csv('../ScrapedData/COdata.csv')

# Scrape ข้อมูล O3

In [ ]:
df_struct = {
    "date":[],
    "provinceNo":[],
    "o3":[]
}

dfO3 = pd.DataFrame(df_struct)
o3Result={}

In [ ]:
initialDate = "20211215"

date = initialDate

while date <= "20220122":
    scrapedValue = findStuff(date,"o3")
    print(scrapedValue)
    o3Result[date] = scrapedValue

    df = pd.DataFrame(
    data=scrapedValue,
    columns=['provinceNo','o3']
    )
    df['date'] = date
    first_column = df.pop('date')
    df.insert(0, 'date', first_column)
    dfO3 = pd.concat([dfO3,df])

    date = nextDay(date)


# Export ข้อมูล O3 ที่ scrape

In [ ]:
dfO3.to_csv('../ScrapedData/O3Data.csv')